In [ ]:
!pip install deepgram-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install pyannote.core
!pip install pyannote.metrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=49e01c8b32165b207529fc3f12186174db61e1fae1785e1c156e115245cb9152
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/AfriSpeech-Dialog-data/data/diarized_tag_afrispeech_dialog_v1.1_49.csv')

In [ ]:
diarize_ds = dataset[dataset['all_timestamps']==True].reset_index(drop=True)

In [ ]:
#remove the 1, 26, 28 rows with incomplete timestamps
diarize_ds = diarize_ds.drop([1, 26, 28]).reset_index(drop=True)
# print(diarize_ds.iloc[28]['transcript'][200:])

In [ ]:
diarize_ds['path'] = diarize_ds['path'].apply(lambda x: x.replace('data/non-medical', '/content/drive/MyDrive/AfriSpeech-Dialog-data/data/non-medical'))
diarize_ds['path'] = diarize_ds['path'].apply(lambda x: x.replace('/data/medical', '/content/drive/MyDrive/AfriSpeech-Dialog-data/data/medical'))

In [ ]:
import getpass
DEEPGRAM_API_KEY = getpass.getpass('Deepgram API Key: ')

Deepgram API Key: ··········


In [ ]:
from deepgram import DeepgramClient, PrerecordedOptions


In [ ]:
diarize_ds.iloc[0]['path']

'/content/drive/MyDrive/AfriSpeech-Dialog-data/data/medical/392c7093-7347-40b8-ab37-db1dcc90945d_4a687442bebec55ed82550bf1617ff4f_tazfKVuj.wav'

In [ ]:
def transcribe(audio_path):
    deepgram = DeepgramClient(DEEPGRAM_API_KEY)

    with open(audio_path, 'rb') as buffer_data:
        payload = { 'buffer': buffer_data }

        options = PrerecordedOptions(
            smart_format=True, model="nova-2", language="en-US", diarize=True
        )

        response = deepgram.listen.prerecorded.v('1').transcribe_file(payload, options)
        # print(response.to_json(indent=4))
        return response

In [ ]:
test_res = transcribe(diarize_ds.iloc[0]['path'])

<ipython-input-19-d677953a3e1e>:11: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v('1').transcribe_file(payload, options)


In [ ]:
type(test_res)

deepgram.clients.listen.v1.rest.response.PrerecordedResponse

In [ ]:
len(test_res['results']['channels'][0]['alternatives'][0]['paragraphs']['paragraphs'])

143

In [ ]:
def extract_segments(data):
    segments = []
    # Loop through the list of dictionaries
    for entry in data:
        start_time = entry['start']  # Start time
        end_time = entry['end']  # End time
        speaker_tag = f"Speaker {entry['speaker']}"  # Speaker tag

        # Append the tuple (start, end, speaker_tag) to the segments list
        segments.append((start_time, end_time, speaker_tag))

    return segments

In [ ]:
extract_segments(test_res['results']['channels'][0]['alternatives'][0]['paragraphs']['paragraphs'])

[(0.88, 1.6999999, 'Speaker 0'),
 (3.84, 7.62, 'Speaker 1'),
 (9.12, 12.179999, 'Speaker 0'),
 (14.205, 16.065, 'Speaker 1'),
 (17.085, 18.305, 'Speaker 0'),
 (19.645, 23.585, 'Speaker 1'),
 (25.724998, 34.28, 'Speaker 0'),
 (36.18, 44.365, 'Speaker 1'),
 (45.705, 46.605003, 'Speaker 0'),
 (47.305, 50.605003, 'Speaker 1'),
 (51.785004, 53.645, 'Speaker 0'),
 (54.905003, 58.19, 'Speaker 1'),
 (59.85, 61.789997, 'Speaker 0'),
 (62.57, 67.149994, 'Speaker 1'),
 (68.815, 69.875, 'Speaker 0'),
 (71.295, 73.155, 'Speaker 1'),
 (74.015, 75.155, 'Speaker 0'),
 (76.095, 79.155, 'Speaker 1'),
 (80.975, 81.875, 'Speaker 0'),
 (83.49, 86.39, 'Speaker 1'),
 (87.649994, 88.869995, 'Speaker 0'),
 (90.049995, 100.645, 'Speaker 1'),
 (101.744995, 103.685, 'Speaker 0'),
 (104.225, 109.205, 'Speaker 1'),
 (110.065, 111.924995, 'Speaker 0'),
 (114.17, 119.63, 'Speaker 1'),
 (121.45, 125.31, 'Speaker 0'),
 (127.85, 129.74501, 'Speaker 1'),
 (130.02501, 130.925, 'Speaker 0'),
 (131.865, 142.285, 'Speaker 1'

In [ ]:
from tqdm import tqdm

In [ ]:
diarize_ds['pred_segments'] = ''
for i, path_ in tqdm(enumerate(diarize_ds['path']), total=len(diarize_ds['path']), desc='Processing'):
  res = transcribe(path_)
  pred_segments = extract_segments(res['results']['channels'][0]['alternatives'][0]['paragraphs']['paragraphs'])
  diarize_ds.at[i, 'pred_segments'] = pred_segments

Processing:   0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-19-d677953a3e1e>:11: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v('1').transcribe_file(payload, options)
Processing: 100%|██████████| 30/30 [05:43<00:00, 11.46s/it]


In [ ]:
import re

In [ ]:
def convert_time_to_seconds(timestamp):
    # Split the timestamp into minutes, seconds, and milliseconds
    minutes, seconds, milliseconds = map(float, timestamp.split(':'))
    # Convert the time to seconds (including fractional part from milliseconds)
    total_seconds = minutes * 60 + seconds + milliseconds / 1000
    return total_seconds


def extract_segments(transcript):
    # Regular expression to match the timestamp and speaker tag
    timestamp_pattern = r'(\d{2}:\d{2}:\d{2})'
    speaker_pattern = r'\[([^\]]+)\]'

    lines = transcript.strip().splitlines()
    segments = []

    start_time = None
    speaker_tag = None

    for i in range(len(lines)):
        if re.match(timestamp_pattern, lines[i]):  # Line is a timestamp
            if start_time and speaker_tag:
                # If we have both start and speaker, the current timestamp is the end time
                end_time = convert_time_to_seconds(lines[i])
                segments.append((start_time, end_time, speaker_tag))
                start_time = None
                speaker_tag = None
            # Set the new start time, converting to seconds
            start_time = convert_time_to_seconds(lines[i])
        elif re.match(speaker_pattern, lines[i]):  # Line contains a speaker tag
            speaker_tag = re.findall(speaker_pattern, lines[i])[0]

    return segments

In [ ]:
#ensure new line before speaker tags
diarize_ds['transcript'] = diarize_ds['transcript'].apply(lambda x: str(x).replace('[', '\r\n['))
diarize_ds['ref_segments'] = diarize_ds['transcript'].apply(lambda x: extract_segments(x))

In [ ]:
diarize_ds.head()

,audio_id,path,transcript,age_group,gender,accent,domain,country,audio_duration,doctor_summary,all_timestamps,pred_segments,ref_segments
0,78fb7265-0373-49d5-b933-c28c95cf3eea,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:01:06\r\n\r\n[Speaker 1]: Are you there?\r\...,26-40,Male,Yoruba,OSCE-Doctor-Patient,NG,906.433991,"Mr. Ade, a 35 year old unmarried Yoruba male t...",True,"[(0.88, 1.6999999, Speaker 0), (3.84, 7.62, Sp...","[(1.006, 1.027, Speaker 1), (4.0, 8.038, Speak..."
1,6d5f1e84-dbc5-42d3-898e-bfe9c2cfe166,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,"00:00:06\r\n\r\n[Speaker 1]: Good evening, I'm...",26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,204.622993,Johnny Dobra is a 35-year-old male who present...,True,"[(0.08, 2.3999999, Speaker 0), (2.72, 6.259999...","[(0.006, 2.028, Speaker 1), (2.049, 6.009, Spe..."
2,ab8fd39d-52bc-48fa-bae9-ff636627f1e5,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:00:08\r\n\r\n[Speaker 1]: My name is Nwachu...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,343.260000,"The patient, a 45 year old female accontant, p...",True,"[(0.08, 2.72, Speaker 0), (2.72, 9.78, Speaker...","[(0.008, 2.04, Speaker 1), (2.049, 5.023, Spea..."
3,5129fd8c-7b8c-4d05-a03a-196bcae4deff,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:00:50\r\n\r\n[Speaker 1]: My name is Chiama...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,337.587982,"John Peter, a 45-year-old construction worker,...",True,"[(0.56, 3.86, Speaker 0), (4.08, 6.58, Speaker...","[(0.05, 3.044, Speaker 1), (4.016, 6.035, Spea..."
4,58efe9ef-6c76-4d17-b1b2-f397552a0c0e,/content/drive/MyDrive/AfriSpeech-Dialog-data/...,00:01:13\r\n\r\n[Speaker 1]: Good day. I am Do...,26-40,Female,Urhobo,OSCE-Doctor-Patient,NG,213.635986,"Peter, a 45-year-old male, presented to the ho...",True,"[(1.04, 7.54, Speaker 0), (8.0, 10.5, Speaker ...","[(1.013, 5.006, Speaker 1), (5.028, 6.028, Spe..."


In [ ]:
from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate

def create_pyannote_annotation(segments_list):
    annotation = Annotation()
    for start, end, speaker_tag in segments_list:
        segment = Segment(start, end)
        annotation[segment] = speaker_tag
    return annotation

der_metric = DiarizationErrorRate()

In [ ]:
for i, text in tqdm(enumerate(diarize_ds['transcript']), total=len(diarize_ds['transcript']), desc="Processing"):
    ref_annotation = create_pyannote_annotation(diarize_ds.iloc[i]['ref_segments'])
    pred_annotation = create_pyannote_annotation(diarize_ds.iloc[i]['pred_segments'])
    der = der_metric(ref_annotation, pred_annotation)
    print(f"DER: {100 * der:.2f}%")
#get abs value for whole dataset
ds_der = abs(der_metric)
print(f"Absolute DER for dataset: {100 * ds_der:.2f}%")

Processing:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pyannote/metrics/utils.py:200: UserWarning: 'uem' was approximated by the union of 'reference' and 'hypothesis' extents.
  warnings.warn(
Processing:   7%|▋         | 2/30 [00:00<00:04,  6.25it/s]

DER: 25.63%
DER: 33.31%
DER: 22.77%


Processing:  23%|██▎       | 7/30 [00:00<00:01, 12.47it/s]

DER: 37.65%
DER: 36.93%
DER: 42.16%
DER: 18.45%
DER: 20.65%


Processing:  50%|█████     | 15/30 [00:00<00:00, 24.99it/s]

DER: 36.93%
DER: 4.80%
DER: 8.99%
DER: 50.50%
DER: 4.99%
DER: 4.75%
DER: 7.70%
DER: 16.68%
DER: 2.85%
DER: 8.35%
DER: 5.88%


Processing: 100%|██████████| 30/30 [00:01<00:00, 26.31it/s]

DER: 1.86%
DER: 4.54%
DER: 12.17%
DER: 3.67%
DER: 5.65%
DER: 14.57%
DER: 3.15%
DER: 7.80%
DER: 18.40%
DER: 4.88%
DER: 48.65%
Absolute DER for dataset: 14.21%


In [ ]:
diarize_ds.to_csv('/content/drive/MyDrive/AfriSpeech-Dialog-data/diarization_results/deepgram_diarization_der_0.1421_30.csv', index=False)